In [ ]:
import os, os.path as op
import shutil
import yaml
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import pandas as pd
import xarray as xr
from wand.image import Image as WImage

## Defining the home and data directories
tutorial_home_dir = os.path.abspath(os.getcwd())
work_dir = os.path.abspath(os.getcwd())
print("Tutorial directory: ", tutorial_home_dir)
print("Work directory: ", work_dir)

# Verifying if RAiDER is installed correctly
try:
    import RAiDER
except:
    raise Exception('RAiDER is missing from your PYTHONPATH')

os.chdir(work_dir)

In [ ]:
if not op.exists(op.join(work_dir, 'data')):
    !unzip -a data.zip

In [ ]:
## make lat lon files
src = op.join(work_dir, 'data', 'los_HR.rdr.geo')
dst = op.join(work_dir, 'data', 'los_HR.rdr.geo.nc')
!gdal_translate {src} {dst} -of netcdf
ds_base   = xr.open_dataset(dst)

Lat, Lon  = np.meshgrid(ds_base.lat.data, ds_base.lon.data)
print (Lat.shape, Lon.shape)

da_lat = xr.DataArray(Lat.T, coords=[Lon[0, :], Lat[:, 0]], dims='lon lat'.split())
da_lon = xr.DataArray(Lon.T, coords=[Lon[0, :], Lat[:, 0]], dims='lon lat'.split())

da_lat.to_netcdf(op.join(work_dir, 'data', 'lat_HR.geo'))
da_lon.to_netcdf(op.join(work_dir, 'data', 'lon_HR.geo'))

### Helpers

In [ ]:
def new_yaml_group(example_yaml, dct_group, dst='tmp.yaml'):
    """ Write a temporary yaml file with the new 'value' for 'key', preserving parms in example_yaml"""
    with open(example_yaml, 'r') as f:
        try:
            params = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            raise ValueError(f'Something is wrong with the yaml file {example_yaml}')

    params = {**params, **dct_group}
    dst = op.join(os.path.dirname(example_yaml), dst)
#     print (params)

    with open(dst, 'w') as fh:
        yaml.dump(params, fh, default_flow_style=False)

    print ('Wrote new cfg file:', dst)
    return dst

In [ ]:
yaml_base = os.path.join(tutorial_home_dir, 'example_yamls', 'raider_example_LA_bbox.yaml')
print(yaml_base)

In [ ]:
# check the base file works
!raider.py {yaml_base}

## Models

In [ ]:
from RAiDER.models.allowed import ALLOWED_MODELS
models = ALLOWED_MODELS.copy()
models.remove('NCMR')
print ('Testing Models:', models)

In [ ]:
## may want to test downloading
shutil.rmtree('weather_files')

In [ ]:
for model in models:
    grp = {'weather_model': model}
    cfg = new_yaml_group(yaml_base, grp)
    print('Running:', model)
    !raider.py {cfg}

## Date Options

In [ ]:
# date start only is default

In [ ]:
# date start and end
grp = {'date_group': {'date_start' : 20200103, 'date_end': 20200107}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

In [ ]:
# date start and end every 2 days
grp = {'date_group': {'date_start' : 20200103, 'date_end': 20200107, 'date_step': 2}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

In [ ]:
# list of dates
grp = {'date_group': {'date_list': [20200110, 20200115]}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

## Time Options

In [ ]:
# start time only is default

In [ ]:
# start and end time
grp = {'time_group': {'time': '00:00:00', 'end_time': '06:00:00'}}
cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

## AOI Options

In [ ]:
# bbox
!raider.py {yaml_base}

In [ ]:
##TODO: Change to ++process

# bbox global for GNSS
grp = {'aoi_group': {'bounding_box': '-90 90 -180 180'}}
cfg = new_yaml_group(yaml_base, grp)

# !raider.py {cfg} --dostep download_gnss

In [ ]:
# geocoded file (GUNW)
prod = 'S1-GUNW-D-R-071-tops-20200130_20200124-135156-34956N_32979N-PP-913f-v2_0_4.nc'
grp  = {'aoi_group': {'geocoded_file': op.join(work_dir, 'data', prod)}}
cfg  = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

In [ ]:
!gdalinfo {work_dir}/GMAO_wet_20200103T000000_ztd.GTiff

In [ ]:
# geocoded file (SRTM DEM)
dem  = op.join(work_dir, 'data', 'SRTM_3arcsec_LA.dem')

grp  = {'aoi_group': {'geocoded_file': dem}}
cfg  = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

In [ ]:
!gdalinfo {work_dir}/GMAO_wet_20200103T000000_ztd.GTiff -stats

In [ ]:
# ISCE lat/lon files (Hampton Roads)
# this downloads the DEM and intersects it

grp = {
    'aoi_group': {
        'lat_file': f'{work_dir}/data/lat_HR.geo',
        'lon_file': f'{work_dir}/data/lon_HR.geo'
    },
    'weather_model': 'ERA5',
    'date_group': {'date_start': '20181113'},
    'time_group': {'time': '23:00:00'}
}

cfg = new_yaml_group(yaml_base, grp)
# !raider.py {cfg}

## store it for use later
# shutil.copy(f'{work_dir}/GLO30_fullres_dem.tif', f'{work_dir}/data/GLO30_fullres_dem_HR.tif')

In [ ]:
# gnss station file
yaml_gnss = yaml_base.replace('bbox', 'GNSS')

!raider.py {yaml_gnss}

## Height Options

In [ ]:
# list of heights
!raider.py {yaml_base}

In [ ]:
# check its not all nan
with xr.open_dataset(f'{work_dir}/GMAO_tropo_20200103T000000_ztd.nc') as ds:
    print (ds['wet'].mean())

In [ ]:
# dem (use lat lon at DEM)
grp  = {'height_group': {'dem': f'{work_dir}/data/SRTM_3arcsec_LA.dem', 'use_dem_latlon': True}}
cfg  = new_yaml_group(yaml_base, grp)

!raider.py {cfg}

In [ ]:
# check its not all nan
!gdalinfo {work_dir}/GMAO_wet_20200103T000000_ztd.GTiff -stats

In [ ]:
# ISCE lat/lonfiles (Hampton Roads)

grp = {
    'aoi_group': {
        'lat_file': f'{work_dir}/data/lat_HR.geo',
        'lon_file': f'{work_dir}/data/lon_HR.geo'
    }, 
    'height_group': {},
    #'height_group': {'dem': f'{work_dir}/data/GLO30_fullres_dem_HR.tif'}, # written above in AOI
    'weather_model': 'ERA5',
    'date_group': {'date_start': '20181113'},
    'time_group': {'time': '23:00:00'}
}

cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

In [ ]:
# check its not all nan
!gdalinfo {work_dir}/ERA5_wet_20181113T230000_ztd.GTiff -stats

In [ ]:
# ISCE lat/lon/hgt files (Hampton Roads)
grp = {
    'aoi_group': {
        'lat_file': f'{work_dir}/data/lat_HR.geo',
        'lon_file': f'{work_dir}/data/lon_HR.geo'
    },
    'height_group': {'height_file_rdr': f'{work_dir}/data/dem_HR.crop'},
    'weather_model': 'ERA5',
    'date_group': {'date_start': '20181113'},
    'time_group': {'time': '23:00:00'}
}

cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

In [ ]:
# check valid
!gdalinfo {work_dir}/ERA5_wet_20181113T230000_ztd.GTiff -stats

## LOS Options

In [ ]:
yaml_orbit = op.join(work_dir, 'example_yamls', 'raider_example_UK-isce.yaml')
orbit_file = op.join(work_dir, 'orbits', 'S1B_OPER_AUX_POEORB_OPOD_20211122T112354_V20211101T225942_20211103T005942.EOF')

In [ ]:
# orbit file, bounding box, height levels, ray_trace
!raider.py {yaml_orbit}

In [ ]:
# check its not all nan
with xr.open_dataset(f'{work_dir}/ERA5_tropo_20211102T062159_ray.nc') as ds:
    print (ds['wet'].mean())

In [ ]:
# orbit file, bounding box, height levels, projection
grp  = {
    'los_group': {
        'ray_trace': False,
        'orbit_file': orbit_file
    }
}
cfg  = new_yaml_group(yaml_orbit, grp)
!raider.py {cfg}

In [ ]:
# check its not all nan
with xr.open_dataset(f'{work_dir}/ERA5_tropo_20211102T062159_std.nc') as ds:
    print (ds['wet'].mean())

In [ ]:
# orbit file, bounding box, NO height levels (so at weather model zlevels)
grp  = {'height_group': {}}
cfg  = new_yaml_group(yaml_orbit, grp)
!raider.py {cfg}

In [ ]:
# check its not all nan
with xr.open_dataset(f'{work_dir}/ERA5_tropo_20211102T062159_ray.nc') as ds:
    print (ds['wet'].mean())

In [ ]:
# orbit file, bounding box, NO height levels (so at weather model zlevels), projection
grp  = {'height_group': {}, 'los_group': {'ray_trace': False}}
cfg  = new_yaml_group(yaml_orbit, grp)
!raider.py {cfg}

In [ ]:
# check its not all nan
with xr.open_dataset(f'{work_dir}/ERA5_tropo_20211102T062159_ztd.nc') as ds:
    print (ds['wet'].mean())

In [ ]:
# orbit file, lat/lon grid, no ray tracing
orbit_file_HR = op.join(work_dir, 'orbits', 'S1A_OPER_AUX_POEORB_OPOD_20181203T120749_V20181112T225942_20181114T005942.EOF')

grp = {
    'aoi_group': {
        'lat_file': f'{work_dir}/data/lat_HR.geo',
        'lon_file': f'{work_dir}/data/lon_HR.geo'
    },
    'height_group': {'dem': f'{work_dir}/data/GLO30_fullres_dem_HR.tif'}, # written above in AOI
    'weather_model': 'ERA5',
    'date_group': {'date_start': '20181113'},
    'time_group': {'time': '23:00:00'},
    'los_group':  {
        'ray_trace': True,
        'orbit_file': orbit_file_HR
    }
}

cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

In [ ]:
!gdalinfo {work_dir}/ERA5_wet_20181113T230000_std.GTiff -stats

In [ ]:
# orbit file, lat/lon grid,  ray tracing
orbit_file_HR = op.join(work_dir, 'orbits', 'S1A_OPER_AUX_POEORB_OPOD_20181203T120749_V20181112T225942_20181114T005942.EOF')

grp = {
    'aoi_group': {
        'lat_file': f'{work_dir}/data/lat_HR.geo',
        'lon_file': f'{work_dir}/data/lon_HR.geo'
    },
    'height_group': {'dem': f'{work_dir}/data/GLO30_fullres_dem_HR.tif'}, # written above in AOI
    'weather_model': 'ERA5',
    'date_group': {'date_start': '20181113'},
    'time_group': {'time': '23:00:00'},
    'los_group':  {
        'ray_trace': True,
        'orbit_file': orbit_file_HR
    }
}

cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

In [ ]:
!gdalinfo {work_dir}/ERA5_wet_20181113T230000_ray.GTiff -stats

## Projection Options

In [ ]:
# os.chdir('UTM_DEM')

In [ ]:
## UTM DEM with WGS Model
# !raider.py raiderDelay_utm_CA-ERA5.yaml

In [ ]:
## UTM DEM with UTM Model
# !raider.py raiderDelay_utm_CA-HRRR.yaml

In [ ]:
## WGS DEM with UTM Model
# !raider.py raiderDelay_geo_CA-HRRR.yaml

In [ ]:
# os.chdir('..')